In [ ]:
import torch
from torchvision import datasets, transforms
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import shutil
import os
from tqdm import tqdm
from PIL import Image, ImageDraw
import random

img_size = 64 # 64 lub 128

In [ ]:
class AddRandomBlackRectangle:
    def __init__(self, min_size, max_size):
        self.min_size = min_size
        self.max_size = max_size

    def __call__(self, img):
        width, height = img.size
        rect_width = random.randint(self.min_size, self.max_size)
        rect_height = random.randint(self.min_size, self.max_size)
        x = random.randint(0, width - rect_width)
        y = random.randint(0, height - rect_height)
        
        # Create a new image with the same size as the original image
        rect_img = Image.new('RGB', img.size, (0, 0, 0))
        
        # Create a new mask image with the same size as the original image
        mask_img = Image.new('L', img.size, 0)
        
        # Draw a rectangle on the new image and the mask
        draw_img = ImageDraw.Draw(rect_img)
        draw_mask = ImageDraw.Draw(mask_img)
        draw_img.rectangle([x, y, x + rect_width, y + rect_height], fill="black")
        draw_mask.rectangle([x, y, x + rect_width, y + rect_height], fill=255)
        
        # Rotate the new image and the mask
        angle = random.randint(0, 90)
        rect_img = rect_img.rotate(angle)
        mask_img = mask_img.rotate(angle)
        
        # Paste the new image onto the original image using the mask
        img.paste(rect_img, (0, 0), mask_img)
        
        return img

In [ ]:
# Define a transform to normalize the data
transform = transforms.Compose([
    AddRandomBlackRectangle(img_size//5, img_size//4),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)), # to nam powinno dać wartości między -1 a 1
])

dataset_path_train = f'Nature/divided_x{img_size}/train'
dataset_path_val = f'Nature/divided_x{img_size}/val'
dataset_path_test = f'Nature/divided_x{img_size}/test'

In [ ]:
train_dataset = datasets.ImageFolder(dataset_path_train, transform=transform)
val_dataset = datasets.ImageFolder(dataset_path_val, transform=transform)
test_dataset = datasets.ImageFolder(dataset_path_test, transform=transform)

print('train', len(train_dataset), 'val:', len(val_dataset), 'test:', len(test_dataset))

In [ ]:
train_dl = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=64, shuffle=True)